# 764 Research Project: 
# Enhancing YOLO Cone Detection in Formula SAE with Super-Resolution via Diffusion Models

## ResShift

### Setup Environment

In [ ]:
# Install Miniconda3
!pip install -q condacolab
import condacolab
condacolab.install_miniconda(prefix='/usr/local')
print('The Miniconda has been installed...')

In [ ]:
import os
os.chdir("/content")
CODE_DIR = "ResShift"
print("Done...")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### Clone Repository

In [ ]:
# Clone Repository
import shutil
from pathlib import Path
if Path(CODE_DIR).exists():
  shutil.rmtree(CODE_DIR)

!git clone https://github.com/zsyOAOA/ResShift.git $CODE_DIR
os.chdir(f"./{CODE_DIR}")
print("Done...")

In [ ]:
"""
Managing environments Anaconda:
https://docs.conda.io/projects/conda/en/4.6.0/user-guide/tasks/manage-environments.html
"""
# Viewing the list of your environments
!conda info --envs
# Updating the environment, this step takes about 20 mins, please be patient.
# conda env update -n base -f /content/ResShift/environment.yml
!pip install -r requirements.txt
print("The environemnt has beed updated...")

### Inference

In [ ]:
import cv2
import matplotlib.pyplot as plt
def display(img1, img2):
  fig = plt.figure(figsize=(25, 10))
  ax1 = fig.add_subplot(1, 2, 1)
  plt.title('Zoomed LR', fontsize=16)
  ax1.axis('off')
  ax2 = fig.add_subplot(1, 2, 2)
  plt.title('ResShift', fontsize=16)
  ax2.axis('off')
  ax1.imshow(img1)
  ax2.imshow(img2)
def imread(img_path):
  img = cv2.imread(img_path)
  if img.ndim > 3:
    img = img[:, :, :3]
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  return img

**Upload images**

In [ ]:
import os

# Define the input and output directories
input_dir = 'testdata/fsoco_sample/'
#output_dir = 'results/fsoco_sample/'
output_dir = '/content/drive/MyDrive/764results'

# Loop through all files in the input directory
for filename in os.listdir(input_dir):
    # Check if the corresponding output .png file already exists in the output directory
    output_png_filename = os.path.splitext(filename)[0] + '.png'
    output_png_path = os.path.join(output_dir, output_png_filename)

    if os.path.exists(output_png_path):
        print(f"Skipping {filename} as inference has already been run.")
        continue

    # Check if the file is a valid image file (adjust according to your file types)
    if filename.endswith('.jpg') or filename.endswith('.png'):
        input_path = os.path.join(input_dir, filename)
        print(f"Running inference on {filename}")
        %system python3 inference_resshift.py -i {input_path} -o {output_dir} --task realsr --scale 4 --version v3
        print(f"Inference completed for {filename}")

## YOLO

In [ ]:
import os
HOME = os.getcwd()
print(HOME)

In [ ]:
%pip install ultralytics supervision roboflow
import ultralytics
ultralytics.checks()

In [ ]:
!pip install roboflow

!mkdir {HOME}/datasets
%cd {HOME}/datasets

from roboflow import Roboflow
rf = Roboflow(api_key="XXXXXXXXXXXX")
project = rf.workspace("cone-detection-zoafa").project("764_cones-juywk")
version = project.version(1)
dataset = version.download("yolov11")

In [ ]:
%cd {HOME}
!yolo task=detect mode=train model=yolo11s.pt data='datasets/data.yaml' epochs=100 imgsz=640 plots=True

In [ ]:
from IPython.display import Image as IPyImage

IPyImage(filename=f'{HOME}/runs/detect/train/results.png', width=600)

In [ ]:
from ultralytics import YOLO

model = YOLO('/content/runs/detect/train/weights/best.pt')  # load a custom model

# Validate the model
metrics = model.val()  # no arguments needed, dataset and settings remembered

In [ ]:
metrics.box.map75